In [1]:
import importlib
import random
import mrcfile
import csv
import math
import numpy as np
from pathlib import Path
import napari
import pokemino_generator as poke

In [4]:
importlib.reload(poke)

<module 'pokemino_generator' from '/Users/anna/Library/CloudStorage/OneDrive-Nexus365/RFI/Code/Pokeminos/pokemino_generator.py'>

In [3]:
def add_label(mrc, param_name, param_value):
    
    """Adds information on rotation angles, shifts along x, y and z to the mrc header."""
    
    label_count = mrc.header.nlabl
    mrc.header.label[label_count] = f'{param_name}={str(param_value)}'
    mrc.header.nlabl += 1

## Example Dataset 1.Tetraminos.

In [7]:
importlib.reload(poke)

tetraminos = [[[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 0, 3]],
              [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1]],
              [[0, 0, 0], [0, 0, 1], [0, 1, 1], [0, 1, 2]], 
              [[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 1]], 
              [[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 0]], 
              [[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]]]

from itertools import product

def upscale_coords(brick_list, scale_factor = 5):
    upscaled_coords = []

    for brick in brick_list:
        upscaled_coords.append(list(product(*[[coord * scale_factor + i for i in range(scale_factor)] for coord in brick])))
    
    flat_list = [num for sublist in upscaled_coords for num in sublist]
    return flat_list

tetraminos = [upscale_coords(a) for a in tetraminos]

tetra_id = ['I', 'O', 'Z', 'T', 'L', 'E']

a = 32

volume_large = poke.Volume((2*a, 3*a, a))

for x in range(len(tetra_id)):
    random.seed()
    volume = poke.Volume((a, a, a))
    pokemino = poke.Pokemino3D(size=4*125, volume=volume, brick_coords=tetraminos[x], positioning=(int(a/2), int(a/2), int(a/2)), algorithm=None)
    pokemino.make_coords_relative_to_centre_of_mass()
    volume.fit_pokemino(pokemino)
    with mrcfile.new(Path(f'datasets/tetraminoes/{tetra_id[x]}.mrc'), overwrite=True) as mrc:
        mrc.set_data(volume.array)

## Example Dataset 2. Pokeminos of differnt sizes.

In [9]:
importlib.reload(poke)

pokemino_names = ['Snorlax', 'Squirtle', 'Dragonite', 'Eevee', 'Squishy', 'Mewtwo', 'Pikachu', 'Greninja']

sizes = [10, 12, 15, 15, 20, 22, 30, 35]

a = 32

random.seed(0)
for x in range(8):
    volume = poke.Volume((a, a, a))
    pokemino = poke.Pokemino3D(size = sizes[x],
                               volume = volume, 
                               seed = pokemino_names[x],
                               crowd_factor = 0.2, 
                               target_ratio = [1, 1, 1], 
                               positioning=(int(a/2), int(a/2), int(a/2)), 
                               algorithm="biased",
                               scale_factor = 4)
    
    volume.fit_pokemino(pokemino)
    with mrcfile.new(Path(f'datasets/pokeminoes/{pokemino_names[x]}.mrc'), overwrite=True) as mrc:
        mrc.set_data(volume.array)

100%|██████████| 30/30 [00:00<00:00, 33.59it/s]


## Example 3. Biased Pokeminos.

In [ ]:
importlib.reload(poke)

pokemino_names = ['Bulbosaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle']

target_ratios = [# Training Pokemon
                     [1, 1, 1], # blob
                     [2, 1, 1], # elongated blob (capsule)
                     [5, 1, 1], # awkward rod 
                     [15, 1, 1], # rod
                     [4, 4, 1], # awkward sheet
                     [8, 3, 1], # sheet
                 # Pokemon for testing
                     [5, 4, 1], # variation on an awkward sheet
                     [6, 1.5, 1]] # variation on an awkward rod 

a = 64

random.seed(0)
for x in range(8):
    volume = poke.Volume((a, a, a))
    pokemino = poke.Pokemino3D(size = 300,
                               volume = volume, 
                               seed = pokemino_names[x],
                               crowd_factor = 0.1, 
                               target_ratio = target_ratios[x], 
                               positioning=(int(a/2), int(a/2), int(a/2)), 
                               algorithm="biased",
                               scale_factor=3)
                               # max_ratio=max_ratio)
    
    #pokemino.upscale_pokemino(3)
    #theta_x, theta_y, theta_z = pokemino.rotate_the_pokemino_3_axes()
    volume.fit_pokemino(pokemino)
    with mrcfile.new(Path(f'datasets/biased_pokeminoes/{pokemino_names[x]}.mrc'), overwrite=True) as mrc:
        mrc.set_data(volume.array)
        mrc.header.label[1] = f"ratio_y_x={str(pokemino.ratio[1])}"
        mrc.header.label[2] = f"ratio_z_x={str(pokemino.ratio[2])}"
        mrc.header.nlabl=3

 79%|███████▉  | 233/295 [00:59<00:35,  1.73it/s]

## Create large volumes with all example Pokeminos

In [11]:
a = 64
big_volume = np.zeros((a*2, a*3, a), dtype=np.float32)

# tetra_id = ['Snorlax', 'Squirtle', 'Dragonite', 'Eevee', 'Squishy', 'Mewtwo', 'Pikachu', 'Greninja'] # pokeminos
# tetra_id = ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Charmander', 'Charmeleon', 'Charizard', 'Squirtle', 'Wartortle'] # biased_pokeminos

for i, mino in enumerate(tetra_id):
    with mrcfile.open(f'../datasets/biased_poke/{mino}.mrc') as mrc:
        big_volume[math.floor(i/3)*a:(math.floor(i/3)+1)*a, (i%3)*a:(i%3+1)*a, 0:a] = mrc.data

with mrcfile.new('../datasets/biased_poke/large_volume.mrc', overwrite=True) as mrc:
    mrc.set_data(big_volume)

In [ ]:
# a = 64

big_volume = np.zeros((a*2, a*4, a), dtype=np.float32)

tetra_id = ['1dfo_11', '1dkg_481', '1gfl_275', 'test/1d5n_1',
            '1h7x_149', '1i5e_36', '1ks2_485', 'test/1ek8_68'
           ]

for i, mino in enumerate(tetra_id):
    with mrcfile.open(Path(f'../datasets/mrcs_3R_0T/{mino}.mrc')) as mrc:
        big_volume[math.floor(i/4)*a:(math.floor(i/4)+1)*a, (i%4)*a:(i%4+1)*a, 0:a] = mrc.data

with mrcfile.new(Path(f'../datasets/large_volume.mrc'), overwrite=True) as mrc:
    mrc.set_data(big_volume)

In [11]:
a = 32

big_volume = np.zeros((a*2, a*4, a), dtype=np.float32)

tetra_id = ['L_889', 'O_892', 'Z_899', 'I_980', 'E_982', 'T_992'] 

for i, mino in enumerate(tetra_id):
    with mrcfile.open(Path(f'../datasets/tetraminos_3R_0T/{mino}.mrc')) as mrc:
        big_volume[math.floor(i/3)*a:(math.floor(i/3)+1)*a, (i%3)*a:(i%3+1)*a, 0:a] = mrc.data
        
with mrcfile.open(Path(f'../datasets/tetraminos_3R_0T/test/EL_149.mrc')) as mrc:
    big_volume[16:48, 96:128, 0:32] = mrc.data

with mrcfile.new(Path(f'../datasets/big_volume'), overwrite=True) as mrc:
    mrc.set_data(big_volume)


## Create test sets

In [9]:
importlib.reload(poke)

seeds = [1, 2, 3, 4, 7, 6]
n_bricks = [3, 7, 12, 20, 20, 40]

max_size = 10
j, k, l = 2, 10, 10 # numbers of polyominos on a regular grid in each dimension

test_volume = poke.Volume((max_size * j, max_size * k, max_size * l))

for x in range (j):
    for y in range (k):
        for z in range (l):
            random.seed()
            no_pokemino=random.randint(0,5)
            pokemino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning = (int(max_size*(x+0.5)), int(max_size*(y+0.5)), int(max_size*(z+0.5))))
            pokemino.rotate_the_block_3_axes()

test_volume.fit_all_pokeminos()
test_volume.create_subpixel_resolution_array(5)
test_volume.save_as_mrcfile(Path(f'Exp1_S_vs_L/test.mrc'))
test_volume.display_in_napari()

In [10]:
with open(Path(f'Exp1_S_vs_L/test_description.mrc'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['seed', 'size', 'x', 'y', 'z'])
    for creature in test_volume.creatures:
        x, y, z = creature.positioning
        writer.writerow([creature.seed, creature.size, x, y, z])

In [2]:
importlib.reload(poke)

seeds = [1, 2, 3, 4, 7, 6]
n_bricks = [3, 7, 12, 20, 20, 40]

n_pokeminos = 200
a_volume = 60 
test_volume = poke.Volume ((a_volume, a_volume, a_volume))

for x in range (n_pokeminos):
    random.seed()
    no_pokemino=random.randint(0,5)
    polyomino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning=(random.randint(0, test_volume.shape[0]-1), random.randint(0, test_volume.shape[1]-1), random.randint(0, test_volume.shape[2]-1)), density=x)
    polyomino.rotate_the_block_3_axes()

test_volume.check_for_pairwise_overlap()
test_volume.fit_all_pokeminos(fit_excluded_volume=True)
test_volume.add_extra_volume_to_remove_cuts()
test_volume.create_subpixel_resolution_array(5)
test_volume.save_as_mrcfile(Path(f'Exp2_S_vs_L_w_Noise/test.mrc'))

TypeError: object of type 'NoneType' has no len()

In [115]:
with open(Path(f'Exp2_S_vs_L_w_Noise/test_description.mrc'), 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['seed', 'size', 'x', 'y', 'z'])
    for creature in test_volume.creatures:
        x, y, z = creature.positioning
        writer.writerow([creature.seed, creature.size, x, y, z])

In [38]:
importlib.reload(poke)

seeds = [1, 2, 3, 4, 5, 6]
n_bricks = [3, 7, 12, 20, 20, 400]

n_pokeminos = 100
a_volume = 300
test_volume = poke.Volume((a_volume, a_volume, a_volume))

for x in range (n_pokeminos):
    random.seed()
    no_pokemino=random.randint(0,5)
    pokemino = poke.Pokemino3D(seeds[no_pokemino], n_bricks[no_pokemino], test_volume, positioning=(random.randint(0, test_volume.shape[0]-1), random.randint(0, test_volume.shape[1]-1), random.randint(0, test_volume.shape[2]-1)), algorithm="clumped", density=1)
    pokemino.upscale_pokemino(5)
    pokemino.rotate_the_pokemino_3_axes()

test_volume.check_for_pairwise_overlap()
test_volume.fit_all_pokeminos()
# test_volume.add_extra_volume_to_remove_cuts()

test_volume.display_in_napari()
#test_volume.save_as_mrcfile(Path(f'Exp3_Custom_Rotations/test.mrc'))

tl, br: [-34. -44.  -3.] [367. 310. 326.]


/Users/anna/OneDrive - Nexus365/RFI/Code/Pokeminos/pokemino_generator.py:236: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  a = np.floor(np.array(pokemino.poke_array.shape) / 2).astype(np.int32)
